<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from sklearn.metrics import confusion_matrix, classification_report, roc_curve
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
import pickle
import plotly.graph_objects as go
import numpy as np

X_train_smt = pickle.load(open("saved_feats/X_train_smt", "rb"))
X_test = pickle.load(open("saved_feats/X_test_c", "rb"))
y_train_smt = pickle.load(open("saved_feats/y_train_smt", "rb"))
y_test = pickle.load(open("saved_feats/y_test_c", "rb"))

In [2]:
scaler = MinMaxScaler()
X_train_smt = scaler.fit_transform(X_train_smt)
X_test = scaler.transform(X_test)

In [3]:
clf_mnb = OneVsRestClassifier(MultinomialNB(
    alpha=1.0e-10, fit_prior=True, class_prior=None))
                         
clf_mnb.fit(X_train_smt, y_train_smt)
pred_mnb = clf_mnb.predict(X_test)
pred_prob_mnb = clf_mnb.predict_proba(X_test)

In [4]:
print('Predicted classes:', clf_mnb.classes_)
print('Average accuracy :', np.mean(pred_mnb == y_test)*100)
print('Train accuracy :', (clf_mnb.score(
    X_train_smt, y_train_smt))*100)
print('Test accuracy :', (clf_mnb.score(
    X_test, y_test))*100)

print('\n CONFUSION MATRIX')
print(confusion_matrix(y_test, pred_mnb))
print('\n CLASSIFICATION REPORT')
print(classification_report(y_test, pred_mnb))

Predicted classes: ['Baja' 'Estable' 'Sube']
Average accuracy : 84.90145035329118
Train accuracy : 93.05178200415534
Test accuracy : 84.90145035329118

 CONFUSION MATRIX
[[  10   45    5]
 [ 171 2270  142]
 [   6   37    3]]

 CLASSIFICATION REPORT
              precision    recall  f1-score   support

        Baja       0.05      0.17      0.08        60
     Estable       0.97      0.88      0.92      2583
        Sube       0.02      0.07      0.03        46

    accuracy                           0.85      2689
   macro avg       0.35      0.37      0.34      2689
weighted avg       0.93      0.85      0.89      2689



In [10]:
fpr = {}
tpr = {}
thresh = {}

fpr[0], tpr[0], thresh[0] = roc_curve(
    y_test, pred_prob_mnb[:, 0], pos_label='Baja')
fpr[1], tpr[1], thresh[1] = roc_curve(
    y_test, pred_prob_mnb[:, 1], pos_label='Estable')
fpr[2], tpr[2], thresh[2] = roc_curve(
    y_test, pred_prob_mnb[:, 2], pos_label='Sube')


# Crear las curvas ROC
trace0 = go.Scatter(x=fpr[0], y=tpr[0], mode='lines', name='Baja vs Rest', line=dict(color='orange'))
trace1 = go.Scatter(x=fpr[1], y=tpr[1], mode='lines', name='Estable vs Rest', line=dict(color='green'))
trace2 = go.Scatter(x=fpr[2], y=tpr[2], mode='lines', name='Sube vs Rest', line=dict(color='blue'))

# Crear el layout
layout = go.Layout(title='Multiclass ROC curve', xaxis=dict(title='False Positive Rate'), yaxis=dict(title='True Positive Rate'))

fig = go.Figure(data=[trace0, trace1, trace2], layout=layout)
fig.show()

In [6]:
# Asumiendo que 'y_test' son tus clases reales y 'pred_mnb' son tus predicciones
correct_counts = [0, 0, 0]
incorrect_counts = [0, 0, 0]
classes = ['Baja', 'Estable', 'Sube']

for i in range(len(y_test)):
    if y_test[i] == pred_mnb[i]:
        correct_counts[classes.index(y_test[i])] += 1
    else:
        incorrect_counts[classes.index(y_test[i])] += 1


print(f"Predicciones correctas: {correct_counts}")
print(f"Predicciones incorrectas: {incorrect_counts}")

Predicciones correctas: [10, 2270, 3]
Predicciones incorrectas: [50, 313, 43]


In [7]:
# Asumiendo que 'correct_counts' y 'incorrect_counts' son tus conteos
classes = ['Baja', 'Estable', 'Sube']

# Crea las barras para las predicciones correctas
fig = go.Figure(data=[
    go.Bar(name='Correct', x=classes, y=correct_counts),
    go.Bar(name='Incorrect', x=classes, y=incorrect_counts)
])
fig.update_layout(barmode='group')

fig.show()